<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/generateCode-codeLlama-gsm8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
import csv

In [3]:
from google.colab import userdata

from huggingface_hub import login
login(token=userdata.get('HUG_TOKEN'))

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')


In [5]:
# tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf", truncation=True)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf", truncation=True)

pipeline = transformers.pipeline(
    "text-generation",
    # model="codellama/CodeLlama-7b-hf",
    model="codellama/CodeLlama-7b-Instruct-hf",
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
dataset = load_dataset("donghuna/gsm8k_with_plan")
dataset

README.md:   0%|          | 0.00/345 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/582k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'plan'],
        num_rows: 1000
    })
})

In [7]:
# import textwrap

def generate_and_tokenize_prompt_for_generate_code(question):
    # plan = f"""{question}"""
    plan = f"""
You are a helpful and expert coding assistant. Your task is to solve math-related problems by generating Python code that produces the correct answer. Write clean, efficient, and well-commented Python code to solve the following problem:

Problem:
{question}

Requirements:
- The code should compute the answer directly.
- Include comments to explain key steps.
- Print the final answer as the output.

Provide only the Python code as the response.

Begin:
"""

    return plan

In [12]:
from tqdm import tqdm


output_file = "generated_code-onlyPrompt.csv"

with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["question", "answer", "solution"])  # CSV 헤더 작성

    # for idx, data in enumerate(dataset["train"]):
    for idx, data in enumerate(tqdm(dataset["train"], desc="Processing Data")):
        question = data["question"]
        answer = data["answer"]

        data_prompt = generate_and_tokenize_prompt_for_generate_code(question)

        input_tokens = tokenizer(data_prompt, return_tensors="pt", truncation=True, max_length=512)
        max_length = input_tokens.input_ids.shape[-1] + 400

        sequences = pipeline(
            data_prompt,
            do_sample=True,
            temperature=0.2,
            top_p=0.9,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2,
            max_length=max_length,
        )

        generated_code = sequences[0]['generated_text']

        keyword = "Begin:"
        if keyword in generated_code:
            generated_code = generated_code.split(keyword, 1)[1].strip()

        # if '```' in generated_code:
        #     generated_code = generated_code.split('```')[0]

        # print(sequences[0]['generated_text'])
        # print(generated_code)
        # print('--------------------')
        # break

        writer.writerow([question, answer, generated_code])


KeyboardInterrupt: 

In [ ]:
# !cp generated_code-agent.csv /content/drive/MyDrive/

import pandas as pd


df = pd.read_csv(output_file)



from datasets import Dataset
from google.colab import userdata

dataset = Dataset.from_pandas(df)

dataset.push_to_hub("donghuna/generated_code-gsm8k-only", token=userdata.get('HUG_TOKEN'))
